In [128]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import pickle
import numpy as np
import pandas as pd
import pickle

from multiprocessing import Process
from multiprocessing import Pool
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [135]:

options = Options()

options.add_argument('--user-data-dir=/Users/Arian_Gh/AppData/Local/Google/Chrome/User Data/') 
options.add_argument('--profile-directory=Profile 6')
options.add_argument("--disable-extensions")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--start-maximized")

options.add_argument('disable-infobars')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--remote-debugging-port=9222")
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

prefs = {
    "profile.default_content_setting_values.images": 2
}
options.add_experimental_option("prefs", prefs)

# Add a preference to disable image loading


capabilities = DesiredCapabilities.CHROME.copy()
capabilities['acceptSslCerts'] = True
capabilities['acceptInsecureCerts'] = True




In [136]:
file_path = 'categories.txt'

categories = []

with open(file_path, 'r') as file:
    for line in file:     
        category = line.strip()
        categories.append(category)

In [138]:
all_details = {}
index = 0
urls = []
for count,category in enumerate(categories):
    
    driver = webdriver.Chrome(options=options , desired_capabilities=capabilities)
    driver.get("https://dir.indiamart.com/search.mp?ss=scrap&prdsrc=1&stype=attr=1|attrS&res=RC4&qu-comp=to")
    
    input_section = driver.find_element(By.XPATH,"//*[@id='search_string']")
    input_section.clear()
    time.sleep(3)
    input_section.send_keys(category)
    time.sleep(5)
    input_section.send_keys(Keys.ENTER)
    time.sleep(4)


    soup_old = ""
    while True:
        print(f'Category:{category}----Category Number:{count+1}----Dict_Length:{len(all_details)}',end='\r')
        html_text = driver.page_source
        soup = BeautifulSoup(html_text,"lxml")

        products = soup.find("div",class_='right-group flx1 df fww pr')
        sections = products.find_all("section")

        for section in sections:
            details={}
            try:
                name = section.find("a",class_="prd-name").text
                price = section.find("span",class_="elps elps2 fs14 tac clr7").text
                url = section.find("a",class_="prd-name")["href"].split(" ")[0]
                if url in urls:
                    continue

                details["name"]= name
                details["price"] = price
                details["url"] = url
                details["category"] = category
                
                urls.append(url)
                all_details[index] = details
                index+=1
            except:
                
                try:
                    name = section.find("span",class_="noPdp prd-name").text
                    price = section.find("span",class_="elps elps2 fs14 tac clr7").text
                    url = None
                    details["name"]= name
                    details["price"] = price
                    details["url"] = url
                    details["category"] = category
                    all_details[index] = details
                    index+=1
                except Exception as e:
                    
                    continue

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.implicitly_wait(10)
        
        if soup_old == soup:
            break
        else:
            
            soup_old = soup
        try:
            btn = driver.find_element(By.XPATH, "//*[@id='app']/div/main/div[4]/div[2]/div[2]/span")

            driver.execute_script("arguments[0].click();",btn)

        except:
            break
    driver.close()

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=116.0.5845.111)
Stacktrace:
	GetHandleVerifier [0x00007FF7F7A252A2+57122]
	(No symbol) [0x00007FF7F799EA92]
	(No symbol) [0x00007FF7F786E3AB]
	(No symbol) [0x00007FF7F786A161]
	(No symbol) [0x00007FF7F785CADF]
	(No symbol) [0x00007FF7F785DE82]
	(No symbol) [0x00007FF7F785CEB8]
	(No symbol) [0x00007FF7F785BFBE]
	(No symbol) [0x00007FF7F785BF63]
	(No symbol) [0x00007FF7F785A9E5]
	(No symbol) [0x00007FF7F785B153]
	(No symbol) [0x00007FF7F786FF4B]
	(No symbol) [0x00007FF7F78DEEF7]
	(No symbol) [0x00007FF7F78C6FDA]
	(No symbol) [0x00007FF7F78DEB82]
	(No symbol) [0x00007FF7F78C6DB3]
	(No symbol) [0x00007FF7F789D2B1]
	(No symbol) [0x00007FF7F789E494]
	GetHandleVerifier [0x00007FF7F7CCEF82+2849794]
	GetHandleVerifier [0x00007FF7F7D21D24+3189156]
	GetHandleVerifier [0x00007FF7F7D1ACAF+3160367]
	GetHandleVerifier [0x00007FF7F7AB6D06+653702]
	(No symbol) [0x00007FF7F79AA208]
	(No symbol) [0x00007FF7F79A

In [ ]:
df = pd.DataFrame().from_dict(all_details).T
df = df.drop_duplicates()
df = df.reset_index()
df.to_csv(index=False)